In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print ('libaries imported')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    conda-4.6.8                |           py36_0         876 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_1         4.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The follow

Scraping and rendering the Wikipedia page into the desired pandas dataframe using BeautifulSoup and pandas functions, respectively

In [4]:
import requests
from bs4 import BeautifulSoup
! pip install lxml

In [45]:
toronto = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(toronto, 'lxml')
article = soup.find('table', class_='wikitable sortable')

table_list = []
for rows in article.find_all('td'):
    row = rows.text
    row = row.replace('\n', '')
    table_list.append(row)

In [39]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
df=[]
df=pd.DataFrame(columns=column_names)

Dropping rows labelled 'Not Assigned', merging neighbourhoods with same postcodes and boroughs with neighbourhoods 'Not assigned'

In [40]:
df.iloc[:,0]=table_list[::3]
df.iloc[:,1]=table_list[1::3]
df.iloc[:,2]=table_list[2::3]
df.replace("Not assigned", np.nan, inplace = True)
df.dropna(subset=["Borough"], axis=0, inplace = True)
df.reset_index(drop=True, inplace=True)

In [49]:
for i in range(0, df.shape[0]):
    if pd.isnull(df.loc[i,'Neighborhood']):
        df.replace(df.loc[i,'Neighborhood'], df.loc[i,'Borough'],inplace=True)
df.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [48]:
df['Neighborhood'] = df[['PostalCode','Borough','Neighborhood']].groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x)) 
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

df.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [44]:
df.shape

(103, 3)

Importing geospatial data and rendering pandas dataframe

In [53]:
Toronto_Coord=df
Toronto_Coord1=pd.read_csv("http://cocl.us/Geospatial_data")
Toronto_Coord[['Latitude', 'Longitude']]=Toronto_Coord1[['Latitude', 'Longitude']]
Toronto_Coord.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.763573,-79.188711
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,43.744734,-79.239476
6,M1B,Scarborough,"Rouge,Malvern",43.727929,-79.262029
7,M3B,North York,Don Mills North,43.711112,-79.284577
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.716316,-79.239476
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.692657,-79.264848


In [54]:
df.shape

(103, 5)

Step 3: Exploring and clustering neighbourhoods in Toronto

Getting coordinates of Toronto

In [56]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


Creating a map of Toronto with neighborhoods superimposed on top.

In [59]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(Toronto_Coord['Latitude'], Toronto_Coord['Longitude'], Toronto_Coord['Borough'], Toronto_Coord['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Visualising neighbourhoods in the Borough of Downtown Toronto

In [61]:
Downtown_data = Toronto_Coord[Toronto_Coord['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.763573,-79.188711
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
4,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259


In [63]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Downtown Toronto are 43.6541737, -79.3808116451341.


In [64]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(Downtown_data['Latitude'], Downtown_data['Longitude'], Downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

Using Foursquare API to cluster and segment neighbourhoods (client ID and secret hidden for privacy purposes)

In [65]:
CLIENT_ID = 'NHAK4YESAV1S5ORCZAUVBQDB4CJMZB2SYJXCHCNYF12VLQD0'
CLIENT_SECRET = 'NOPEVERSION = '20190315'

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: IREALLYDONTWANTYOUTOKNOWCLIENT_SECRET:IHF5N0XIS4F0Y1UQSEH0ADWUYOWZKTCPVWQECYTZN0O2ZZ1S


Getting neighbourhood names and their coordinates

In [67]:
Downtown_data.loc[0, 'Neighborhood']

'Harbourfront,Regent Park'

In [69]:
neighborhood_latitude = Downtown_data.loc[0, 'Latitude'] 
neighborhood_longitude = Downtown_data.loc[0, 'Longitude']

neighborhood_name = Downtown_data.loc[0, 'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront,Regent Park are 43.7635726, -79.1887115.


Getting top 100 venues in Harbourfront within a radius of 500 meters

Creating url and checking results using get request

In [96]:
limit=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=NHAK4YESAV1S5ORCZAUVBQDB4CJMZB2SYJXCHCNYF12VLQD0&client_secret=IHF5N0XIS4F0Y1UQSEH0ADWUYOWZKTCPVWQECYTZN0O2ZZ1S&v=20190315&ll=43.7635726,-79.1887115&radius=500&limit=100'

In [97]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8b3ae9f594df7cdec9a4b1'},
 'response': {'headerLocation': 'Scarborough Village',
  'headerFullLocation': 'Scarborough Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.768072604500006,
    'lng': -79.18249216787879},
   'sw': {'lat': 43.7590725955, 'lng': -79.1949308321212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b6074e3f964a5200fe729e3',
       'name': 'Swiss Chalet Rotisserie & Grill',
       'location': {'address': '4410 Kingston Rd',
        'lat': 43.76769708292701,
        'lng': -79.1899135003439,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.76769708292701,
          'lng': -79.1899135003439}],
        'distance': 469

In [98]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Cleaning json and rendering pandas dataframe

In [103]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.long']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(30)

,name,categories,lat,long
0,Swiss Chalet Rotisserie & Grill,Pizza Place,43.767697,NaN
1,G & G Electronics,Electronics Store,43.765309,NaN
2,Marina Spa,Spa,43.766000,NaN
3,Big Bite Burrito,Mexican Restaurant,43.766299,NaN
4,Enterprise Rent-A-Car,Rental Car Location,43.764076,NaN
5,Woburn Medical Centre,Medical Center,43.766631,NaN
6,Eggsmart,Breakfast Spot,43.767800,NaN


In [101]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.
